# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document

## Idan Anav



## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
#import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\Anav\Desktop\אדריאן\5


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [24]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [25]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [26]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

### Cleaning the data


In [10]:
df_train['story']= df_train['story'].str.replace('[^\u0590-\u05FF]', ' ')  
df_train = df_train.drop_duplicates(subset ="story", keep= "first")

### Splitting the data


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_train.story, df_train.gender, test_size = 0.15, random_state = 51, shuffle=False)


### Using pipline to test different models

In [12]:
ml_models={
    'DecisionTree'      : DecisionTreeClassifier(max_depth=15),
    'SGDClassifier'     : SGDClassifier(loss='hinge', penalty='l1', alpha=1e-4, random_state=42, max_iter=50, tol=None),
    'Naive Bayes'       : MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
    'LinearSVC'         : LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
    intercept_scaling=1, loss='squared_hinge', max_iter=1000,
    multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
    verbose=0),
    'Perceptron'        : Perceptron(),
    'KNN'               : KNeighborsClassifier(n_neighbors=9, metric='manhattan'),

}

In [53]:
from sklearn.metrics import make_scorer

for model_name in ml_models.keys():
    #the cleaning lambda -> removing all the words that are not hebrew or their length is 1 
    classifier = Pipeline([ ('vect', TfidfVectorizer(ngram_range=(1, 2), tokenizer=lambda x: [word_token[1] for word_token in ht.tokenize(x) if len(word_token[1]) > 1])),
                           ('clf', ml_models[model_name])])
    
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring=make_scorer(f1_score, average='macro'))

    # print the cross-validation scores
    print('Cross-validation scores for', model_name, ':', scores)
    print('Mean F1 score for', model_name, ':', np.mean(scores))
    print()

Cross-validation scores for DecisionTree : [0.58709677 0.43929912 0.57747573 0.46666667 0.55408874 0.52776699
 0.50797963 0.64646465 0.63265306 0.61926769]
Mean F1 score for DecisionTree : 0.5558759045338714

Cross-validation scores for SGDClassifier : [0.7586727  0.54285714 0.77631068 0.80693816 0.60816327 0.67066895
 0.70885176 0.60625    0.55944056 0.72190153]
Mean F1 score for SGDClassifier : 0.6760054748332552

Cross-validation scores for Naive Bayes : [0.43362832 0.43362832 0.43362832 0.43362832 0.43362832 0.43362832
 0.43243243 0.43243243 0.43243243 0.43243243]
Mean F1 score for Naive Bayes : 0.43314996412341544

Cross-validation scores for LinearSVC : [0.43362832 0.42857143 0.43362832 0.5        0.43362832 0.43362832
 0.43243243 0.43243243 0.49886364 0.43243243]
Mean F1 score for LinearSVC : 0.4459245636568646

Cross-validation scores for Perceptron : [0.74358974 0.47805825 0.65548654 0.81714286 0.6768932  0.61212121
 0.63970588 0.62574257 0.60109751 0.70747151]
Mean F1 score f

#### I chose to test  SGDClassifier  and Perceptron  because they got the best scores
#### I used param_grid for that

In [18]:
#SGDClassifier pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=(1, 2), tokenizer=lambda x: [details[1] for details in ht.tokenize(x) if len(details[1]) > 1])),
    ('clf', SGDClassifier(loss='hinge', penalty='l1', alpha=1e-4, random_state=42, max_iter=50, tol=None))
])

#The different data i wanna test
param_grid = {
    'vect__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [1e-3, 1e-4, 1e-5],
    'clf__penalty': ['l1', 'l2'],
    'clf__max_iter': [50, 100, 200]
}

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=10, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        tokenizer=<function <lambda> at 0x0000018336B5FC10>)),
                                       ('clf',
                                        SGDClassifier(max_iter=50, penalty='l1',
                                                      random_state=42,
                                                      tol=None))]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.001, 0.0001, 1e-05],
                         'clf__max_iter': [50, 100, 200],
                         'clf__penalty': ['l1', 'l2'],
                         'vect__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best parameters:  {'clf__alpha': 0.0001, 'clf__max_iter': 200, 'clf__penalty': 'l1', 'vect__max_df': 0.75}
Best score:  0.6945038101768455


In [19]:
#Perceptron pipeline

clf = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=lambda x: [word_token[1] for word_token in ht.tokenize(x) if len(word_token[1]) > 1])),
    ('clf', Perceptron())
])

param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'clf__alpha': [1e-2, 1e-3, 1e-4],
    'clf__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__max_iter': [10, 50, 100]
}



grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10, scoring='f1_macro')

grid_search.fit(X_train, y_train)

# print the best hyperparameters and the corresponding f1 score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best F1 Score: ", grid_search.best_score_)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function <lambda> at 0x000001833D801700>)),
                                       ('clf', Perceptron())]),
             param_grid={'clf__alpha': [0.01, 0.001, 0.0001],
                         'clf__max_iter': [10, 50, 100],
                         'clf__penalty': ['l1', 'l2', 'elasticnet'],
                         'vect__ngram_range': [(1, 1), (1, 2), (2, 2)]},
             scoring='f1_macro')

Best Hyperparameters:  {'clf__alpha': 0.0001, 'clf__max_iter': 10, 'clf__penalty': 'elasticnet', 'vect__ngram_range': (1, 2)}
Best F1 Score:  0.6438974289404177


### It seems that the SGDClassifier is the better option

In [20]:
clf = Pipeline([
     #the cleaning lambda -> removing all the words that their length is 1 
    #min_df lowered my score, max featured also lowered my score
    ('vect', TfidfVectorizer(ngram_range=(1, 2), tokenizer=lambda x: [details[1] for details in ht.tokenize(x) if len(details[1]) > 1], max_df=0.75,)),
    ('clf', SGDClassifier(loss='hinge', penalty='l1', alpha=1e-4, random_state=42, max_iter=200, tol=None))
])
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [21]:
from sklearn.metrics import make_scorer
# perform cross-validation
scores = cross_val_score(clf, X_train, y_train, cv=10, scoring=make_scorer(f1_score, average='macro'))

# print the cross-validation scores
print('Cross-validation scores:', scores)
print('Mean F1 score:', np.mean(scores))

Cross-validation scores: [0.77142857 0.63954944 0.81714286 0.75478927 0.63817759 0.6479648
 0.66373626 0.58111702 0.70923077 0.72190153]
Mean F1 score: 0.6945038101768455


### testing my model on the first 5 and last 5 from df_story

In [52]:
# Loading my trained model 
model = clf

for i in range(5):
    # Cleaning and tokenizing the stories from the beginning  
    cleaned_story = df_test.story[i].replace('[^\u0590-\u05FF]', ' ')
    tokenized_story = ' '.join([details[1] for details in ht.tokenize(cleaned_story) if len(details[1]) > 1])

    # Cleaning and tokenizing the stories from the end  
    cleaned_back_story = df_test.story.iloc[-(i+1)].replace('[^\u0590-\u05FF]', ' ')
    tokenized_back_story = ' '.join([details[1] for details in ht.tokenize(cleaned_back_story) if len(details[1]) > 1])
    
    # Make the prediction using the trained model
    predicted_gender = model.predict([tokenized_story])
    Back_predicted_gender = model.predict([tokenized_back_story])

    print(f"Predicted gender in the {i} spot:{predicted_gender}   Predicted gender in the {len(df_test.index)-i-1} spot:", Back_predicted_gender)

Predicted gender in the 0 spot:['m']   Predicted gender in the 322 spot: ['m']
Predicted gender in the 1 spot:['m']   Predicted gender in the 321 spot: ['m']
Predicted gender in the 2 spot:['m']   Predicted gender in the 320 spot: ['m']
Predicted gender in the 3 spot:['m']   Predicted gender in the 319 spot: ['m']
Predicted gender in the 4 spot:['f']   Predicted gender in the 318 spot: ['m']


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [28]:
df_test['story']= df_test['story'].str.replace('[^\u0590-\u05FF]', ' ')  
df_test = df_test.drop_duplicates(subset ="story", keep= "first")

In [39]:
test_prediction = clf.predict(df_test.story)

In [40]:
df_predicted = pd.DataFrame({"test_example_id":df_test.test_example_id,"predicted_category":test_prediction})

In [41]:
df_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)